#Data Cleaning

In [ ]:
#load in necessary packages
!pip install git+https://github.com/cinemagoer/cinemagoer
import numpy as np
import os
import pandas as pd
import statistics
import time
import traceback
import sys
import datetime
import pytz
import pandas_gbq

#Cinemagoer needs to be capitalized on import
import imdb
from imdb import Cinemagoer as cg, IMDbError
from imdb.helpers import sortedEpisodes

#creating instance of Cinemagoer
ia = imdb.Cinemagoer()

In [ ]:
#load in data
path = 'drive/MyDrive/Data Sci/NFLX - What We Watched/'
os.listdir(path)

nflx = pd.read_csv(path+'What_We_Watched_A_Netflix_Engagement_Report_2023Jan-Jun.csv', header=4)

#Data Cleaning

In [ ]:
#rename columns for readability
nflx = nflx.rename(columns={'Available Globally?': 'avail_globally', 'Release Date': 'release_date', 'Effective Release Date': 'eff_release_date', 'Hours Viewed': 'hours_viewed'})

In [ ]:
#convert columns into datetime
nflx['release_date'] = pd.to_datetime(nflx['release_date'])
nflx['eff_release_date'] = pd.to_datetime(nflx['eff_release_date'])

#convert hours viewed to integer
nflx['hours_viewed'] = pd.to_numeric(nflx['hours_viewed'].astype(str).str.replace(',', ''))

In [ ]:
#creating columns for IMDB search
nflx['movieID'] = pd.NA
nflx['imdb_title'] = pd.NA

In [ ]:
#creating columns that identify the title of the show and the season seperately
nflx['true_title'] = nflx['Title'].str.extract(r'^(.*?)(: Season|: Limited Series|: Part|: Chapter|: Book|: Volume|: Series (\d+))')[0].str.strip()
nflx['Season/Series/Part'] = nflx['Title'].str.extract(r'(: Season (\d+)|: Part (\d+)|: Chapter (\d+)|: Book(\d+)|: Series (\d+)|: Volume (\d+))').bfill(axis=1).iloc[:, 1].fillna(0).astype(int)

#IMDb ID Scraping

In [ ]:
#top 500 titles (out of 18,000) account for approx. 43% of all hours viewed
#93.5 billion hours viewed in total
#top 500 is responsible for just under 40b hours viewed
#top 715 is responsible for just about 50% of hours
#top 2175 is responsible for 75% of hours
a = nflx['hours_viewed'].iloc[0:500].sum()
b = nflx['hours_viewed'].iloc[:].sum()
(a/b)*100

42.88771518331778

In [ ]:
#obtain the IMDb id for each title

#load in top 500 if it hasn't already been scraped
try:
    nflx_top_500 = pd.read_csv(path+'netflix-top-500.csv')
except FileNotFoundError:
    nflx_top_500 = pd.DataFrame()

if nflx_top_500.empty:

  #gathering IMDb IDs (& titles, according to IMDb) for top 100 titles
  nflx_top_500 = nflx[['Title', 'avail_globally', 'release_date', 'true_title',	'Season/Series/Part', 'movieID',	'imdb_title']].iloc[0:500]

  #setting the titles and IDs as variables for the loops
  df = nflx_top_500
  titles = df['true_title']
  IDs = df['movieID']

  #find the IMDb id for each title
  for title in titles:
    try:
      digits = ia.search_movie(title)[0].movieID
    except IndexError:
      #create a null if IMDb record can't be located
      digits = pd.NA
    #save the IMDb id
    df.loc[df['true_title'] == title, 'movieID'] = digits

  #find the title in IMDb to check for mismatches
  for ID in IDs:
    try:
      cleaned_ID = ID.lstrip('0')  # Remove leading zeros from IMDb id
    except AttributeError:
      cleaned_ID = pd.NA

    try:
        example = ia.get_movie(cleaned_ID)['title']
    except (TypeError, IMDbDataAccessError):
        example = pd.NA

    #save the title as according to IMDb
    if pd.notna(example):
        df.loc[df['movieID'] == ID, 'imdb_title'] = example
    else:
        df.loc[df['movieID'] == ID, 'imdb_title'] = pd.NA

  #check mismatches of titles to determine if Cinemagoer has scraped the wrong title
  df.loc[df['true_title'] != df['imdb_title'], ['Title', 'movieID', 'imdb_title', 'true_title']]

In [ ]:
#manually filling titles with missing IDs
df.loc[df['true_title'].str.contains('Demon Slayer'), 'movieID'] = 9335498
df.loc[df['true_title'].str.contains('Demon Slayer: Kimetsu no Yaiba: Tanjiro Kamado'), 'Season/Series/Part'] = 1

#Hunger // คนหิว เกมกระหาย = 22695402
df.loc[df['true_title'].str.contains('Hunger // คนหิว เกมกระหาย'), 'movieID'] = 22695402

#All Quiet on the Western Front // Im Westen... is 1016150
df.loc[df['true_title'].str.contains('All Quiet on the Western Front //'), 'movieID'] = 1016150

#ONE PIECE: East Blue is 11757066
df.loc[df['true_title'].str.contains('ONE PIECE: East Blue'), 'movieID'] = 11757066

#Boys Over Flowers: Volume 1 is 1370334
df.loc[df['true_title'].str.contains('Boys Over Flowers'), 'movieID'] = 1370334
df.loc[df['true_title'].str.contains('Boys Over Flowers'), 'Season/Series/Part'] = 1

#Through My Window 2: Across the Sea is 21245882
df.loc[df['true_title'].str.contains('Through My Window 2: Across the Sea'), 'movieID'] = 21245882

#Unlocked // 스마트폰을 떨어뜨렸을 뿐인데 is 26160190
df.loc[df['true_title'].str.contains('Unlocked // 스마트폰을 떨어뜨렸을 뿐인데'), 'movieID'] = 26160190

#Mother's Day // Dzień Matki is 19724192
df.loc[df['true_title'].str.contains("Mother's Day // Dzień Matki"), 'movieID'] = 19724192

#Rebelde Way (2002): Temporada 1 is 0328789 (and is Season 1)
df.loc[df['true_title'].str.contains('Rebelde Way'), 'Season/Series/Part'] = 1

In [ ]:
#grabbing titles for projects that previously had missing IDs
IDs = df.loc[df['imdb_title'].isna(), 'movieID']

for ID in IDs:
    try:
      cleaned_ID = ID.lstrip('0')  # Remove leading zeros
    except AttributeError:
        cleaned_ID = pd.NA

    try:
        example = ia.get_movie(cleaned_ID)['title']
    except (TypeError, IMDbDataAccessError):
        example = pd.NA

    if pd.notna(example):
        df.loc[df['movieID'] == ID, 'imdb_title'] = example
    else:
        df.loc[df['movieID'] == ID, 'imdb_title'] = pd.NA

In [ ]:
  #check mismatches of titles to determine if Cinemagoer has scraped the wrong title
  df.loc[df['true_title'] != df['imdb_title'], ['Title', 'movieID', 'imdb_title', 'true_title']]

,Title,movieID,imdb_title,true_title
17,BEEF: Season 1,14403178,Beef,BEEF
22,Extraction 2,12263384,Extraction II,Extraction 2
24,Doctor Cha: Limited Series // 닥터 차정숙: 리미티드 시리즈,27075857,Doctor Cha Jeong Suk,Doctor Cha
34,"Pablo Escobar, el patrón del mal: Season 1",2187850,Pablo Escobar: El Patrón del Mal,"Pablo Escobar, el patrón del mal"
44,Mr. Queen // 철인왕후,13400006,Mr. Queen,Mr. Queen // 철인왕후
...,...,...,...,...
476,Shameless (U.S.): Season 4,1586680,Shameless,Shameless (U.S.)
480,The Mitchells vs. The Machines,7979580,The Mitchells vs the Machines,The Mitchells vs. The Machines
482,All For Love: Season 1 // Amar y vivir: Tempor...,10347886,All for Love,All For Love (2020)
489,Thicker Than Water: Season 1 // Jusqu'ici tout...,27124268,Thicker Than Water,Thicker Than Water (2023)


In [ ]:
#correcting titles that Cinemagoer has misidentified

df.loc[df['true_title'] == 'Til Money Do Us Part', 'true_title'] = 'Until Money Do Us Part'

df.loc[df['true_title'].str.contains('Woman of the Dead'), 'true_title'] = 'Woman of the Dead (2022)'

df.loc[df['true_title'].str.contains('Hajime no Ippo'), 'movieID'] = 481256

df.loc[df['true_title'].str.contains('Sky High: The Series'), 'true_title'] = 'Sky High (2023)'

df.loc[df['true_title'].str.contains('Paw Patrol: The Movie'), 'true_title'] = 'Paw Patrol: The Movie (2021)'

df.loc[df['true_title'] == ('Trolls'), 'true_title'] = 'Trolls (2016)'

df.loc[df['true_title'].str.contains('The Queen of Flow'), 'true_title'] = 'The Queen of Flow (2021) (2018)'

df.loc[df['true_title'].str.contains('Masha and the Bear'), 'true_title'] = 'Masha and the Bear (2007)'

df.loc[df['true_title'] == 'The Nurse', 'true_title'] = 'The Nurse (2023)'

df.loc[df['true_title'] == 'Home', 'true_title'] = 'Home (2015)'

df.loc[df['true_title'] == ('Faraway'), 'movieID'] = 18747542

df.loc[df['true_title'].str.contains('Newly Rich, Newly Poor'), 'true_title'] = 'Newly Rich, Newly Poor (2007)'

df.loc[df['true_title'].str.contains('JUNG_E'), 'true_title'] = 'JUNG_E'

df.loc[df['true_title'] == 'Puss in Boots', 'true_title'] = 'Puss in Boots (2011)'

df.loc[df['true_title'].str.contains('In Love All Over Again'), 'true_title'] = 'In Love All Over Again (2023)'

df.loc[df['true_title'].str.contains('Pokémon The Series: Indigo League'), 'true_title'] = 'Pokémon (1997)'

df.loc[df['true_title'].str.contains('Demon Slayer: Kimetsu no Yaiba: Swordsmith Village Arc'), 'Season/Series/Part'] = 4

df.loc[df['true_title'].str.contains('Pokémon Journeys: The Series'), 'true_title'] = 'Pokémon Journeys'

df.loc[df['true_title'].str.contains('Sanctuary: Season 1'), 'true_title'] = 'Sanctuary (2023)'

df.loc[df['true_title'].str.contains('As Aventuras de Poliana'), 'true_title'] = 'As Aventuras de Poliana (2018)'

df.loc[df['true_title'].str.contains('All For Love'), 'true_title'] = 'All For Love (2020)'

df.loc[df['true_title'].str.contains('Thicker Than Water'), 'true_title'] = 'Thicker Than Water (2023)'

In [ ]:
#generating nulls for titles without IMDb pages
mask = df['true_title'].notna() & df['true_title'].str.contains("Blippi's Adventures", na=False)
df.loc[mask, ['movieID', 'imdb_title']] = pd.NA

mask = df['true_title'].notna() & df['true_title'].str.contains('Little Angel', na=False)
df.loc[mask, ['movieID', 'imdb_title']] = pd.NA

In [ ]:
#rerunning the adjusted titles through the IMDb id scraper
adjusted_titles = [
    'Until Money Do Us Part',
    'Woman of the Dead (2022)',
    'Sky High (2023)',
    'Paw Patrol: The Movie (2021)',
    'Trolls (2016)',
    'The Queen of Flow (2021) (2018)',
    'Masha and the Bear (2007)',
    'The Nurse (2023)',
    'Home (2015)',
    'Newly Rich, Newly Poor (2007)',
    'JUNG_E',
    'Puss in Boots (2011)',
    'In Love All Over Again (2023)',
    'Pokémon (1997)',
    'Pokémon Journeys',
    'Sanctuary (2023)',
    'As Aventuras de Poliana (2018)',
    'All For Love (2020)',
    'Thicker Than Water (2023)'
]

for title in adjusted_titles:
    try:
        digits = ia.search_movie(title)[0].movieID
    except IndexError:
        digits = pd.NA
    df.loc[df['true_title'] == title, 'movieID'] = digits

#excluding rows with nulls and selecting rows where the title from data and IMDb title are mismatched
valid_rows = df.dropna(subset=['true_title', 'imdb_title'])
for index, row in valid_rows[valid_rows['true_title'] != valid_rows['imdb_title']].iterrows():
    movieID = row['movieID']

    try:
        cleaned_ID = movieID.lstrip('0')  # Remove leading zeros
    except AttributeError:
        cleaned_ID = pd.NA

    try:
        example = ia.get_movie(cleaned_ID)['title']
    except (TypeError, IMDbDataAccessError):
        example = pd.NA

    if pd.notna(example):
        df.at[index, 'imdb_title'] = example
    else:
        df.at[index, 'imdb_title'] = pd.NA

In [ ]:
#selecting rows with a missing IMDb title and excluding Blippi's Adventures & Little Angel
valid_rows = df.loc[df['imdb_title'].isna() & ~df['true_title'].str.contains("Blippi's Adventures|Little Angel", case=False, na=False)]
IDs = valid_rows['movieID']

for index, ID in IDs.iteritems():
    try:
        cleaned_ID = str(int(ID)).lstrip('0')  # Convert to integer, remove leading zeros
    except (AttributeError):
        cleaned_ID = None

    try:
        example = ia.get_movie(cleaned_ID)['title']
    except (TypeError, IMDbDataAccessError):
        example = None

    if pd.notna(example):
        df.at[index, 'imdb_title'] = example
    else:
        df.at[index, 'imdb_title'] = pd.NA

In [ ]:
#merge the new dataset with netflix top 500
merged_df = pd.merge(nflx, df, on='Title', how='inner', suffixes=('_nflx', '_df'))

In [ ]:
columns_to_drop = ['movieID_nflx', 'imdb_title_nflx', 'true_title_nflx', 'Season/Series/Part_nflx', 'avail_globally_df', 'release_date_df']

# Drop the duplicated columns
merged_df.drop(columns=columns_to_drop, inplace=True)

In [ ]:
columns_to_rename = {
    'avail_globally_nflx': 'avail_globally',
    'release_date_nflx': 'release_date',
    'true_title_df': 'true_title',
    'Season/Series/Part_df': 'Season_Series_Part',
    'movieID_df': 'movieID',
    'imdb_title_df': 'imdb_title'
}

# Rename columns for readability
merged_df.rename(columns=columns_to_rename, inplace=True)

In [ ]:
#save merged data as csv
merged_df.to_csv('nflx-top-500.csv')

#IMDb 'Kind' Scrape

In [ ]:
#set temporary dataframe
xdf = nflx_top_500[['Title', 'Season_Series_Part', 'movieID']]

#drop titles with missing IDs
xdf = xdf.dropna(subset=['movieID'])

#convert IDs to integers
xdf['movieID'] = xdf['movieID'].astype(int)

In [ ]:
#obtaining all 'kinds' of titles for filtering

key = 'kind'

IDs = xdf.loc[xdf['kind'].isna(), 'movieID']

error_IDs = []

error_IDs_2 = []

for ID in IDs:
    try:
      title = ia.get_movie(ID)

      #create a boolean mask based on movieID
      mask = xdf['movieID'] == ID

      #save the 'kind' of title to the dataframe for each row
      value = title[key]
      xdf.loc[mask, key] = value

    except IMDbError:
      #save IDs with errors in list for rechecking
      error_IDs.append(ID)

#rerun IDs that got errors in
for ID in error_IDs:
    try:
        title = ia.get_movie(ID)

        #create a boolean mask based on movieID
        mask = xdf['movieID'] == ID

        #save the 'kind' of title to the dataframe for each row
        value = title[key]
        xdf.loc[mask, key] = value

    except IMDbError:
        #save IDs with a 2nd error in list for rechecking
        error_IDs_2.append(ID)

In [ ]:
#retrieve every unique kind of title
xdf['kind'].unique()

In [ ]:
#save title kinds to csv
xdf.to_csv('title-kinds.csv')

#IMDb Primary Scrape

In [ ]:
#list important data for scraping
main_keys = [
    'aspect ratio',
    'country codes',
    'distributors',
    'genres',
    'imdbID',
    'kind',
    'language codes',
    'production companies',
    'rating',
    'runtimes',
    'seasons',
    'stars',
    'year',
]


#list kinds of TV listed in IMDb for filtering
tv_types = [
    'tv series', 'tv mini series'
]

In [ ]:
#create DataFrame for full scrape
title_kinds = pd.read_csv(path+'title-kinds.csv')
df = title_kinds
df = df.rename(columns={'Season_Series_Part': 'Season'})

#converting Season to integer
df['Season'] = df['Season'].astype(int)

#properly numbering mini-series with no 'Season' #
df.loc[(df['kind'] == 'tv mini series') & (df['Season'] == 0), 'Season'] = 1

#The Lorax is wrong, it should be the 2012 movie, its actual ID is #1482459
df.loc[(df['Title'] == "Dr. Seuss' The Lorax"), 'movieID'] = '1482459'

#initialize columns for keys
for key in main_keys:
    df[key] = 0

#initialize episode count:
df['eps_in_season'] = 0

#initialize error counter columns
error_counters = [
    'runtime_key_error',
    'runtime_timeout_error',
    'rating_key_error',
    'rating_timeout_error',
    'other_errors',
    'general_timeout_error'
]

for error_counter in error_counters:
    df[error_counter] = 0

In [ ]:
#function to simultaneously log messages/errors while presenting them in output
def custom_print(message, log_file):
    print(message)
    print(message, file=log_file)

#function to handle exceptions
def handle_exception(title, ID, e, log_file):
    print(f"Error: {e} for {title} (ID: {ID})", file=log_file)
    print(f"Exception details: {repr(e)}", file=log_file)
    traceback.print_exc(file=log_file)
    time.sleep(delay_seconds)

#function to handle timeouts and notify in output & log simultaneously
def handle_timeout_error(title, ID, te, log_file):
    custom_print(f"Timeout Error for {title} (ID: {ID})", log_file)
    print(f"Exception details: {repr(te)}", file=log_file)
    time.sleep(delay_seconds)

In [ ]:
#Scrape of all necessary IMDb data as identified in main_keys
#Uses batch processing and checkpoints to save progress in a csv for every 100 titles processed
#prints status and certain errors while simultaneously saving them in a log file

max_retries = 2
delay_seconds = 5
batch_size = 25
checkpoint_interval = 100

#set path for saving data
path2 = 'drive/MyDrive/Data Sci/NFLX - What We Watched/Scrape/'

#set variables for loop
df_titles = df['Title']
IDs = df['movieID']

#set the Pacific Timezone
pacific = pytz.timezone('America/Los_Angeles')
#get the current time in Pacific Time
now_pacific = datetime.datetime.now(pacific)
#format the timestamp for readability
timestamp = now_pacific.strftime("%Y_%m_%d_%I_%M%p")

#log all prints and errors for analysis and debugging
log_file_path = (path2 + f'output_log_{timestamp}.txt')
with open(log_file_path, 'w') as log_file:

  batch_counter = 0  #initialize batch counter

  for start in range(0, len(df_titles), batch_size):
      batch_titles = df_titles[start:start + batch_size]
      batch_IDs = IDs[start:start + batch_size]

      #track processing
      now_pacific = datetime.datetime.now(pacific)
      timestamp = now_pacific.strftime("%Y_%m_%d_%I_%M%p")
      custom_print(f"Processing Batch: {start + 1} - {min(start + batch_size, len(df_titles))} at {timestamp}", log_file)

      for ID, df_title in zip(batch_IDs, batch_titles):
          #initializing variables in case they are needed in TV episode loop
          trt = 0
          sum_ratings = 0
          ratings_count = 0
          _ep_runtime_count = 0
          _ep_rating_count = 0

          #initializing error counters
          other_errors_count = 0
          general_timeout_error_count = 0
          runtime_key_error = 0
          runtime_timeout_error = 0
          rating_key_error = 0
          rating_timeout_error = 0

          #retry loop in case of TimeoutErrors
          for retry in range(max_retries):
              try:
                title = ia.get_movie(ID)

                #create a boolean mask based on movieID
                mask2 = df['Title'] == df_title

                for key in main_keys:
                    if key in title:
                      value = title[key]

                      if isinstance(value, (list, tuple)):
                        #if iterable, concatenate the list into a single string and update DataFrame
                        concatenated_value = ', '.join(map(str, value))
                        df.loc[mask2, key] = concatenated_value
                      else:
                        #if not iterable, update dataframe with the single value
                        df.loc[mask2, key] = value

                #obtaining individual episode data for each season of TV
                if not df.loc[mask2 & (df['kind'].isin(tv_types))].empty:

                  #update the title object with the episodes infoset
                  ia.update(title, 'episodes')

                  #obtain array of episodes in season
                  eps = sortedEpisodes(title, season=int(df.loc[mask2, 'Season'].iloc[0]))
                  print(f"Number of episodes: {len(eps)}")
                  #save number of episodes
                  eps_count = int(len(eps))
                  df.loc[mask2, 'eps_in_season'] = eps_count

                  #getting runtimes for each episode to calc total runtime
                  for ep in eps:
                    _ep = ia.get_movie(ep.movieID)
                    try:
                      mins = int(_ep.get('runtimes', [0])[0])
                      trt += mins
                      _ep_runtime_count += 1
                    except KeyError as k:
                      #log and count key errors
                      handle_exception(df_title, ID, k, log_file)
                      runtime_key_error += 1
                    except TimeoutError as te:
                      #log and count timeout errors
                      handle_timeout_error(df_title, ID, te, log_file)
                      runtime_timeout_error += 1

                  df.loc[mask2, 'runtime_key_error'] = runtime_key_error
                  df.loc[mask2, 'runtime_timeout_error'] = runtime_timeout_error

                  #only replacing the 'runtime' column if each episode has a valid 'runtime' value. otherwise leave the runtime initially retrieved by the movieID
                  if (df.loc[mask2, 'runtime_key_error'].iloc[0] == 0) & (_ep_runtime_count == eps_count) & (_ep_runtime_count != 0):
                    df.loc[mask2, 'runtimes'] = trt
                  print(f"{df_title} has TRT of {df.loc[mask2, 'runtimes'].iloc[0]}", file=log_file)
                  else:
                      # Handle the case where no individual episode runtimes were available. leave the rating first retrieved by the movieID
                      print(f"{df_title} has incomplete individual episode runtimes", file=log_file)

                  #getting ratings for each episode to calc average
                  for ep in eps:
                    _ep = ia.get_movie(ep.movieID)
                    try:
                      ep_rating = float(_ep.get('rating', 0))
                      sum_ratings += ep_rating
                      ratings_count += 1
                    except KeyError as k:
                      #log and count key errors
                      handle_exception(df_title, ID, k, log_file)
                      rating_key_error += 1
                    except TimeoutError as te:
                      #log and count timeout errors
                      handle_timeout_error(df_title, ID, te, log_file)
                      rating_timeout_error += 1

                  df.loc[mask2, 'rating_key_error'] = rating_key_error
                  df.loc[mask2, 'rating_timeout_error'] = rating_timeout_error

                  #only replacing the 'rating' column if each episode has a valid 'rating' value. otherwise leave the rating initially retrieved by the movieID
                  if (df.loc[mask2, 'rating_key_error'].iloc[0] == 0) & (ratings_count == eps_count) & (_ep_rating_count != 0):
                      average_rating = sum_ratings / ratings_count
                      df.loc[mask2, 'rating'] = average_rating
                      print(f"{df_title} has rating of {df.loc[mask2, 'rating'].iloc[0]}", file=log_file)
                  else:
                      # Handle the case where no individual episode ratings were available. leave the rating first retrieved by the movieID
                      print(f"{df_title} has incomplete individual episode ratings", file=log_file)

                break  #break the retry loop if successful

              #log and count any other unexpected errors
              except Exception as e:
                handle_exception(df_title, ID, e, log_file)
                other_errors_count += 1
              except TimeoutError as te:
                handle_timeout_error(df_title, ID, te, log_file)
                general_timeout_error_count += 1

          df.loc[mask2, 'other_errors'] = other_errors_count
          df.loc[mask2, 'general_timeout_error'] = general_timeout_error_count

          #increase batch counter for every successful loop
          batch_counter += 1

          #save data every 100 titles with a timestamped title
          if batch_counter % checkpoint_interval == 0:
              now_pacific = datetime.datetime.now(pacific)
              timestamp = now_pacific.strftime("%Y_%m_%d_%I_%M%p")
              checkpoint_number = batch_counter // checkpoint_interval
              checkpoint_filename = f'scrape_checkpoint_{checkpoint_number}_{timestamp}.csv'
              df.to_csv(path2 + checkpoint_filename, index=False)
              custom_print(f"Checkpoint {checkpoint_number} reached at {timestamp}", log_file)

#save all data if end of dataframe is successfully reached
timestamp_final = now_pacific.strftime("%Y_%m_%d_%I_%M%p")
df.to_csv(path2 + f'scrape-final_{timestamp_final}.csv', index=False)

#Importing to BigQuery

In [ ]:
path2 = 'drive/MyDrive/Data Sci/NFLX - What We Watched/Send to Big Query/'
os.listdir(path2)

#setting up BigQuery project ID and dataset for export
project = 'netflix-what-we-watched'
dataset = 'top_500'

#importing each dataset from Google Sheets
sheets = ['hours_viewed',
          'title_kinds',
          'imdb_data_full'
]

for sheet in sheets:
  table = pd.read_csv(path2+f'{sheet}.csv', header=1)

  # send each dataframe to the corresponding table in BigQuery
  pandas_gbq.to_gbq(df, f'{dataset}.{table}', project_id=project)